<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js043_H2MoleculeLDA3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [1]:
#@title js043_H2MoleculeLDA3D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023-2024 Mitsuru Ikeuchi
#  home page: https://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.09.30 created,  last updated on 2024.05.21
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js043_H2MoleculeLDA3D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js043_H2MoleculeLDA3D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.06.20 created, last updated on 2018.11.20
//    ver 0.0.1  2019.01.17 v1, last updated on 2021.06.03
//    ver 0.0.2  2021.11.01 v2, last updated on 2021.11.01
//    ver 0.0.3  2023.03.26 v3, last updated on 2023.08.24
//
// -------------------- RS-DFT - local density approximation 3D
//
// - real space density functional theory - local density approximation
// - solve Kohn-Sham equation - successive approximation
// - Vxc : LDA(local density approximation)
//         J. P. Perdew and A. Zunger; Phys. Rev., B23, 5048 (1981)
//
//  many electron system --> one electron approximation
//    one electron ,  others -> consider as effective potential Veff
//    Kohn-Sham equation {(-d^2/dr^2)+Veff(r)} |i> = e_i |i>,  Veff(r)=Vext+VH+Vx+Vc
//      Veff: one electron feels effective potential
//      Vext: external Potential
//      VH  : Hartree potential (electron charge -> electro-static potential)
//      Vx,Vc :electron exchange and correlation potential (LDA:Perdew-Zunger)
//    successive approximation
//      solve Kohn-Sham -> electron state,occupation -> electron charge density -> Veff
//
//  procedure
//   (1) given: trial |i>, occupation(i)
//
//   (2) set electron density rho
//       rho <-- |i>, occupation[(i), mixing rho(iter-1)
//
//   (3) set effective potential
//        Veff = Vext + VH + Vx + Vc
//        VH <-- rho (Poisson eq. ,SOR iteration)
//        Vx,Vc <-- rho (LDA:Perdew-Zunger)
//
//   (4) solve Kohn-Sham equation (successive approximation)
//        |i> steepest descent method: |i(next)> = |i> - dump{H-E}|i>
//        E(i) <-- <i|H|i>
//        {|0>,..,|i>,..,|N>} orthogonallization : Gram-Schmidt
//
//   (5) sort state
//        sort orbit by E(i)
//
//   (6) set occupation
//        occupation(i) <-- E(i)
//
//   (7) goto (2)
//
//  result - total energy E(au) - ppDistance d(au)
//    ppDistance d(au)    0.5     1.0     1.5     2.0     2.5
//    this calc. E(au)   -0.297  -0.984  -1.073  -1.059  -1.024
//    experimental E=-1.17au minimum at d=1.4au
//
// --------------------
*/

const H2MoleculeLDA3D = (function(){ // ====================  H2MoleculeLDA3D Module  ====================

	const g_auLength = 5.29177211e-11;		// (m) 1(au) = auLength (m), (au: atomic unit hBar=1,e=1,me=1,a0=1)
	const g_auTime = 2.418884326e-17;		// (s) 1(au) = auTime (s)
	const g_auEnergy = 4.35974465e-18;		// (J) 1(au) = auEnergy (J)
	const g_au2eV = 27.211386;				// (eV) 1(au) = 27.211386 (eV)
	const g_nMax = 64;						// maximum of NNx or NNy or NNz
	const g_stMax = 10;						// maximum number of state
	const g_nucMax = 20;					// maximum number of nucleus

	let g_iterCount = 0;					// LDA iteration count
	let g_numberOfElectron = 2.0;			// numberOfElectron = zNuc + dzNuc
	let g_nucCharge = 2.0;					// H+ + H+ = 2.0
	let g_numberOfOrbit = 6;				// number of orbit
	let g_NNx = 36;							// number of space x-division, x-BoxSize = NNx*dx
	let g_NNy = 32;							// number of space y-division
	let g_NNz = 32;							// number of space z-division
	let g_dx = 1.0/2.0;						// (au) x-division
	let g_dy = g_dx;						// (au) y-division
	let g_dz = g_dx;						// (au) z-division
	let g_energyMem = 0.0;					// temporal memory of energy
	let g_iterationError = 1.0;				// energy difference for iteration n and n+1
	let g_dampingFactor = 0.2*g_dx*g_dx;	// damping factor of steepest descent method (dt = 0.5*dx*dx)
	let g_mixing = 0.5;						// charge mixing in setRho()
	let g_broadening = 0.01;				// (au) level broadening IN setOccupation
	let g_ppDistance = 1.5;					// (au) p-p distance = dNuc*dx, dNuc = 1,2,3,4,5

	const g_sdEnergy = dim1( g_stMax );		// sdEnergy[stMax] electron orbit energy
	const g_occupation = dim1( g_stMax );	// occupation[stMax] of orbit
	const g_sdState = dim4( g_stMax, g_nMax ); // sdState[stMax][nMax][nMax][nMax] electron state 0...19
	const g_wrk = dim3( g_nMax );			// wrk[nMax][nMax][nMax] state work space in steepestDescent
	const g_vv = dim3( g_nMax );			// vv[nMax][nMax][nMax] effective potential
	const g_vvext = dim3( g_nMax );			// external potential
	const g_vvh = dim3( g_nMax );			// Hartree potential
	const g_vvx = dim3( g_nMax );			// exchange potential
	const g_vvc = dim3( g_nMax );			// correlation potential
	const g_rho = dim3( g_nMax );			// electron charge density

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dim3( n ) {
		let a = [];
		for (let i=0; i<n; i++) {
			a[i] = [];
			for (let j=0; j<n; j++) {
				a[i][j] = new Float64Array( n );
			}
		}
		return a;
	}

	function dim4( nst, n ) {
		let a = [];
		for (let i=0; i<nst; i++) {
			a[i] = dim3( n );
		}
		return a;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( stateMax, dNuc, dne ) {
		g_numberOfOrbit = stateMax;
		setNumberOfElectron(dne);
		setInitialState(stateMax);
		setPotential(dNuc);
	}

	function setNumberOfElectron(dne) {
		g_numberOfElectron = g_nucCharge + dne;
	}

	function setInitialState(stateMax) {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;
		clearSdState(stateMax);
		for (let ist=0; ist<stateMax; ist++) {
			for (let i=1; i<nnx-1; i++) {
				for (let j=1; j<nny-1; j++) {
					for (let k=1; k<nnz-1; k++) {
						g_sdState[ist][i][j][k] = Math.random()-0.5;
					}
				}
			}
			normalizeState(ist);
		}
	}

	function clearSdState(stMax) {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;
		for (let ist=0; ist<stMax; ist++) {
			for (let i=0; i<nnx; i++) {
				for (let j=0; j<nny; j++) {
					for (let k=0; k<nnz; k++) {
						g_sdState[ist][i][j][k] = 0.0;
					}
				}
			}
		}
	}

	function setPotential(dNuc) {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dx=g_dx, dy=g_dy, dz=g_dz;
		g_ppDistance = dNuc;
		g_iterCount = 0;
		const x0 = 0.5*nnx*dx-1.0, y0 = 0.5*nny*dy, z0 = 0.5*nnz*dz;
		const x1 = 0.5*nnx*dx-1.0+g_ppDistance, y1 = 0.5*nny*dy, z1 = 0.5*nnz*dz;
		for (let i=0; i<nnx; i++) {
			const x = i*dx;
			for (let j=0;j<nny;j++) {
				const y = j*dy;
				for (let k=0;k<nnz;k++) {
					const z = k*dz;
					let r0 = Math.sqrt((x-x0)*(x-x0)+(y-y0)*(y-y0)+(z-z0)*(z-z0));
					if (r0<0.5*dx) r0 = 0.5*dx;
					let r1 = Math.sqrt((x-x1)*(x-x1)+(y-y1)*(y-y1)+(z-z1)*(z-z1));
					if (r1<0.5*dx) r1 = 0.5*dx;
					g_vvext[i][j][k] = -1.0/r0 -1.0/r1 ;
					g_vvh[i][j][k] = -g_vvext[i][j][k];

				}
			}
		}
	}


	// --------------------  iterate LDA  --------------------

	function iterateLDA( iterMax, mix, broad ) {
		const errorDecisionOrbit = Math.floor((g_numberOfElectron-1.0)/2.0);
		g_mixing = mix;
		g_broadening = broad;
		setElectronDensity(g_numberOfOrbit);
		setEffectivePotential();
		solveKohnSham(g_numberOfOrbit,iterMax);
		sortState(g_numberOfOrbit);
		setOccupation(g_numberOfOrbit,g_numberOfElectron);
		g_iterationError = g_sdEnergy[errorDecisionOrbit] - g_energyMem;
		g_energyMem = g_sdEnergy[errorDecisionOrbit];
	}

	// --- (2) set electron density rho -from sdState[], occupation[]

	function setElectronDensity(stateMax) {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					g_rho[i][j][k] *= (1.0-g_mixing);
					for (let ie=0; ie<stateMax; ie++) {
						if (g_occupation[ie]<=0.0) continue;
						g_rho[i][j][k] += g_mixing*g_occupation[ie]*(g_sdState[ie][i][j][k]*g_sdState[ie][i][j][k]);
					}
				}
			}
		}
	}

	// --- (3) set effective potential -from electron density

	function setEffectivePotential() {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;
		poisson(20); // setVh
		setVxc();
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					g_vv[i][j][k] = g_vvext[i][j][k]+g_vvh[i][j][k]+g_vvx[i][j][k]+g_vvc[i][j][k];
				}
			}
		}
	}

	function poisson(iterMax) { // solve (d^2/dr^2) V = 4 pai rho
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, h2=4.0*Math.PI*g_dx*g_dx;
		const w = (1.0/6.0)*1.8; // 1/6 * SOR omega(1.0<omega<2.0)
		for (let iter=0; iter<iterMax; iter++) {
			for (let i=1; i<nnx-1; i++) {
				for (let j=1; j<nny-1; j++) {
					for (let k=1; k<nnz-1; k++) {
						g_vvh[i][j][k] += w*(g_vvh[i+1][j][k]+g_vvh[i-1][j][k]+g_vvh[i][j+1][k]+g_vvh[i][j-1][k]
										+g_vvh[i][j][k+1]+g_vvh[i][j][k-1]-6.0*g_vvh[i][j][k] +h2*g_rho[i][j][k]);
					}
				}
			}
		}
	}

	// LDA :  J. P. Perdew and A. Zunger; Phys. Rev., B23, 5048 (1981)
	function setVxc() {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;
		const c1 = -0.984745022;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					const rh = g_rho[i][j][k];
					const rh3 = Math.pow(rh,0.33333333);
					g_vvx[i][j][k] = c1*rh3;
					const rs = 0.6204/(rh3+1.0e-20);
					if (rs>=1.0) {
						const sqrtrs = Math.sqrt(rs);
						const ec = -0.1423/(1.0+1.0529*sqrtrs+0.3334*rs);
						g_vvc[i][j][k] = ec*(1.0+1.22838*sqrtrs+0.4445*rs)/(1.0+1.0529*sqrtrs+0.3334*rs);
					} else {
						g_vvc[i][j][k] = -0.05837-0.0084*rs +(0.0311+0.00133*rs)*Math.log(rs);
					}
				}
			}
		}
	}

	function eeCorrelation(rh) {
		const r = 0.6204/(Math.pow(rh,0.33333333)+1.0e-20);
		let ec;
		if (r>=1.0) {
			ec = -0.1423/(1.0+1.0529*Math.sqrt(r)+0.3334*r);
		} else {
			ec = -0.0480-0.0116*r+(0.0311+0.0020*r)*Math.log(r);
		}
		return ec;
	}

	// --- (4) solve Kohn-Sham equation

	function solveKohnSham(stateMax, iterMax) {
		for (let i=0; i<iterMax; i++) {
			for (let ist=0; ist<stateMax; ist++) {
				g_sdEnergy[ist] = steepestDescent(ist, g_dampingFactor);
			}
			GramSchmidt(stateMax);
			g_iterCount += 1;
		}
	}

	function steepestDescent(ist,damp) {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, h2=2*g_dx*g_dx, p = g_sdState[ist];
		const ei = energyOfState(ist);
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					const pijk = p[i][j][k];
					g_wrk[i][j][k] = (6*pijk-p[i+1][j][k]-p[i-1][j][k]-p[i][j+1][k]-p[i][j-1][k]
									-p[i][j][k+1]-p[i][j][k-1])/h2 + (g_vv[i][j][k]-ei)*pijk;
				}
			}
		}
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					g_sdState[ist][i][j][k] -= damp*g_wrk[i][j][k];
				}
			}
		}
		normalizeState(ist);
		return ei;
	}

	function energyOfState(ist) {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, h2=2*g_dx*g_dx, p = g_sdState[ist];;
		let s = 0.0, sn = 0.0;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					const pijk = p[i][j][k];
					s += pijk*( (6*pijk-p[i+1][j][k]-p[i-1][j][k]-p[i][j+1][k]-p[i][j-1][k]
						-p[i][j][k+1]-p[i][j][k-1])/h2 +　g_vv[i][j][k]*pijk);
					sn += pijk*pijk;
				}
			}
		}
		return s/sn;
	}

	function GramSchmidt(stateMax) {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;
		normalizeState(0);
		for (let istate=1; istate<stateMax; istate++) {
			for (let ist=0; ist<istate; ist++) {
				const s = innerProduct(ist,istate);
				for (let i=0; i<nnx; i++) {
					for (let j=0; j<nny; j++) {
						for (let k=0; k<nnz; k++) {
							g_sdState[istate][i][j][k] -= s*g_sdState[ist][i][j][k];
						}
					}
				}
			}
			normalizeState(istate);
		}
	}

	// --- (5) sort state

	function sortState(stateMax) {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;
		for (let ist=stateMax-2; ist>=0; ist--) {
			if (g_sdEnergy[ist]>g_sdEnergy[ist+1]+0.00001) {
				for (let i=0; i<nnx; i++) {
					for (let j=0; j<nny; j++) {
						for (let k=0; k<nnz; k++) {
							const w = g_sdState[ist][i][j][k];
							g_sdState[ist][i][j][k] = g_sdState[ist+1][i][j][k];
							g_sdState[ist+1][i][j][k] = w;
						}
					}
				}
				const w = g_sdEnergy[ist];
				g_sdEnergy[ist] = g_sdEnergy[ist+1];
				g_sdEnergy[ist+1] = w;
			}
		}
	}

	// --- (6) set occupation

	function setOccupation(maxState, nElectron) {
		let eUpper = g_sdEnergy[maxState-1]+1.0;
		let eLower = g_sdEnergy[0]-1.0;
		for (let i=0; i<maxState; i++) {
			if(g_sdEnergy[i]>eUpper) eUpper = g_sdEnergy[i];
			if(g_sdEnergy[i]<eLower) eLower = g_sdEnergy[i];
		}
		while (eUpper-eLower>1.0e-12) {
			const eFermi = (eUpper+eLower)/2.0;
			const ntrial = trialOcc(maxState, eFermi);
			if (ntrial<nElectron) {
				eLower = eFermi;
			} else {
				eUpper = eFermi;
			}
		}
		const eFermi = (eUpper+eLower)/2.0;
		for (let i=0; i<maxState; i++) {
			g_occupation[i] = 2.0*FermiDirac(g_sdEnergy[i], eFermi);
			if (g_occupation[i]<0.0001) g_occupation[i] = 0.0;
			if (2.0-g_occupation[i]<0.0001) g_occupation[i] = 2.0;
		}
	}

	function trialOcc(maxState,eFermi) {
		let s = 0.0;
		for (let i=0;i<maxState;i++) {
			s += 2.0*FermiDirac(g_sdEnergy[i], eFermi);
		}
		return s;
	}

	function FermiDirac(ee, ef) {
		const et = g_broadening; //levelWidth
		return ( 1.0/(Math.exp((ee-ef)/et)+1.0) );
	}


	// --------------------  utility  --------------------

	function innerProduct(ist,jst) {
		const nnx=g_NNx ,nny=g_NNy ,nnz=g_NNz;
		let s = 0.0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					s += g_sdState[ist][i][j][k]*g_sdState[jst][i][j][k];
				}
			}
		}
		return s*g_dx*g_dy*g_dz;
	}

	function normalizeState(ist) {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;
		let s = 0.0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					s += g_sdState[ist][i][j][k]*g_sdState[ist][i][j][k];
				}
			}
		}
		const a = Math.sqrt(1/(s*g_dx*g_dy*g_dz));
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					g_sdState[ist][i][j][k] = a*g_sdState[ist][i][j][k];
				}
			}
		}
	}

	function totalEnergy() {
		const nOrbit=g_numberOfOrbit, nnx=g_NNx, nny=g_NNy, nnz=g_NNz;

		const enn = 1.0/g_ppDistance;

		let sei = 0.0;
		for (let i=0; i<nOrbit; i++) {
			sei += g_occupation[i]*g_sdEnergy[i];
		}

		let s = 0.0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					s += (-0.5*g_vvh[i][j][k]-0.25*g_vvx[i][j][k]
						+eeCorrelation(g_rho[i][j][k])-g_vvc[i][j][k])*g_rho[i][j][k];
				}
			}
		}
		s = s*g_dx*g_dy*g_dz;

		return enn+sei+s;
	}

	function totalElectronCharge() {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;
		let s = 0.0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					s += g_rho[i][j][k];
				}
			}
		}
		return s*g_dx*g_dy*g_dz;
	}


	// --------------------  cloud  --------------------

	const g_cloud = dim3(g_NNx,g_NNy,g_NNz);
	const g_srnd = [];

	function setCloud(iState) {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dv=g_dx*g_dy*g_dz;
		set_srnd();
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					g_cloud[i][j][k] = 0;
				}
			}
		}
		let s = 0, ip = 0
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					const stijk = g_sdState[iState][i][j][k];
					const dens = stijk*stijk;
					s += dens*dv;
					while (s>g_srnd[ip] && ip<1000) {
						g_cloud[i][j][k] += 1;
						ip += 1;
					}
				}
			}
		}
	}

	function set_srnd() {
		g_srnd[0] = Math.random();
		for (let i=1; i<1001; i++) {
			g_srnd[i] = g_srnd[i-1] + Math.random();
		}
		for (let i=0; i<1000; i++) {
			g_srnd[i] = g_srnd[i]/g_srnd[1000];
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( stateMax, dNuc, dne )
		evolve:			iterateLDA,				// iterateLDA( iterMax, mix, broad )

		setNumberOfElectron: setNumberOfElectron,	// setNumberOfElectron( dne );
		setPotential:	setPotential,			// setPotential( dNuc )
		setCloud:		setCloud,				// setCloud( ist )

		getAUinSI:		function() { return [ g_auLength, g_auTime, g_auEnergy, g_au2eV ]; },
		getSysParam:	function() { return [ g_NNx, g_NNy, g_NNz, g_dx, g_nucCharge ]; },
		getNow:			function() { return [ g_iterCount, totalEnergy() ]; },
		getOccupation:	function(ist) { return g_occupation[ist]; },
		getStEnergy:	function(ist) { return g_sdEnergy[ist]; },
		getState:		function(ist,i,j,k) { return g_sdState[ist][i][j][k]; },
		getDensity:		function(i,j,k) { return g_rho[i][j][k]; },
		getVext:		function(i,j,k) { return g_vvext[i][j][k]; },
		getVeff:		function(i,j,k) { return g_vv[i][j][k]; },
		getVh:			function(i,j,k) { return g_vvh[i][j][k]; },
		getVx:			function(i,j,k) { return g_vvx[i][j][k]; },
		getVc:			function(i,j,k) { return g_vvc[i][j][k]; },
		getCloud:		function(i,j,k) { return g_cloud[i][j][k]; },
	};

})(); // ====================  H2MoleculeLDA3D end  ====================


const js043 = (function(){ // ====================  js Module  ====================

	const theModule = H2MoleculeLDA3D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;					// canvas2d
	let ctx;

	const v_stateMax = 6;
	let v_dNuc = 1.5;		// (au) distance of H - H
	let v_dne = 0.0;		// -1.0:+ion,  -0.5:ionization,  0.0:atom, +1.0:-ion
	let v_iterMax = 1;		// iterateLDA(iterMax)
	let v_mix = 0.5;		// mixing electron density: rhoNext = (1-mix)*rho + mix*sum( occupation[i]*|orbit[i]|^2 )
	let v_broad = 0.01;		// (au) level broadening (0.01au ~ 300K room temperature)

	let p_NNx, p_NNy, p_NNz, p_dx, p_nucCharge; // = theModule.getSysParam();
	let auLength, auTime, auEnergy, au2eV; // = theModule.getAUinSI();
  let iterCount, totalEnergy; // = theModule.getNow();
	let nowData = [];
  let occList = [];
  let energyList = [];

	let vextArray = [];
	let veffArray = [];
  let densityArray = [];
	let orb0Array = [];
	let orb1Array = [];
	let orb2Array = [];
	let orb3Array = [];

	let dispMode = 4;
	let dispOrbit = 0;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
	let inStepFlag = false;
	let potentialChanged = false;

  let breakFlag = false;
  let getFieldFlag = true;
  let fieldKind = 1;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();
		viewHome();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g3d.setMouseOnCanvas( canvas ); // 3D graphics
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_stateMax, v_dNuc, v_dne );
			[ p_NNx, p_NNy, p_NNz, p_dx, p_nucCharge ] = theModule.getSysParam();
			//g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
			g3d.init( p_NNx, p_NNy, p_NNz, p_dx, xCanvasSize, yCanvasSize, 300, 20 );
			g3d.drawField3D.threshold = 1.0;
			getFieldFlag = true;
			fieldKind = 1;
		}

		if ( potentialChanged ) {
			potentialChanged = false;
			theModule.setPotential( v_dNuc );
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_iterMax, v_mix, v_broad );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_iterMax, v_mix, v_broad );
			inStepFlag = true;
		}

		draw( ctx, dispMode );

		if ( getFieldFlag ) setFieldData( fieldKind );

		requestAnimationFrame(animate);
	}

  function setFieldData( fieldKind ) {
    for (let ist=0; ist<v_stateMax; ist++) {
      occList[ist] = theModule.getOccupation(ist);
      energyList[ist] = theModule.getStEnergy(ist);
    }

    if (fieldKind==1) {
			nowData = [ iterCount, totalEnergy, occList, energyList ];
			vextArray = [];
			veffArray = [];
			densityArray = [];
			orb0Array = [];
			orb1Array = [];
			orb2Array = [];
			orb3Array = [];
      for (let i=0; i<p_NNx; i++) {
				vextArray[i] = [];
				veffArray[i] = [];
        densityArray[i] = [];
				orb0Array[i] = [];
				orb1Array[i] = [];
				orb2Array[i] = [];
				orb3Array[i] = [];
        for (let j=0; j<p_NNy; j++) {
					vextArray[i][j] = [];
					veffArray[i][j] = [];
          densityArray[i][j] = [];
					orb0Array[i][j] = [];
					orb1Array[i][j] = [];
					orb2Array[i][j] = [];
					orb3Array[i][j] = [];
          for (let k=0; k<p_NNz; k++) {
						vextArray[i][j][k] = theModule.getVext(i,j,k);
						veffArray[i][j][k] = theModule.getVeff(i,j,k);
            densityArray[i][j][k] = theModule.getDensity(i,j,k);
						orb0Array[i][j][k] = theModule.getState(0,i,j,k);
						orb1Array[i][j][k] = theModule.getState(1,i,j,k);
						orb2Array[i][j][k] = theModule.getState(2,i,j,k);
						orb3Array[i][j][k] = theModule.getState(3,i,j,k);
          }
        }
      }
    }
  }


	// --------------------  draw  --------------------

	const gColor = { orb:"#dddd00", dens:"#dd88dd",
					Vext:"#00dd00", Veff:"#0088ff", Vh:"#4444ff", Vxc:"#8800ff", Vx:"#aa00ff",Vc:"#8888ff" };

	function draw( ctx, dispMode ) {
		const NNx = p_NNx, NNy = p_NNy, NNz = p_NNz, dx = p_dx, jc = NNy/2, kc = NNz/2;
		const emag = 1.0, rmag = 20.0, pmag = 10.0;
		const xp = 30, yp = 20, xtabp = 320;

		[ auLength, auTime, auEnergy, au2eV ] = theModule.getAUinSI();
		[ iterCount, totalEnergy ] = theModule.getNow();

		// clear
		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		if (dispMode==0) { // 1D density(x,0) + Vext,Veff,Vh,Vxc
			draw1D( ctx, 0, xp, yp );
		} else if (dispMode==1) { // 1D orbit(x,0) + Vext
			draw1D( ctx, 1, xp, yp );

		} else if (dispMode==2) { // 2D density(x,y)
			draw2D( ctx, -1, xp, yp);
			ctx.fillText("density(x,y)", xp, yCanvasSize-10);
		} else if (dispMode==3) { // 2D orbit(x,y)
			draw2D( ctx, dispOrbit, xp, yp);
			ctx.fillText("orbit(x,y)", xp, yCanvasSize-10);

		} else if (dispMode==4) { // 3D - density(x,y,z)
			dispText( "density(x,y,z)" );
			g3d.drawField3D.threshold = 0.5;
			const densFunc = function(i,j,k) { return 100.0*theModule.getDensity(i,j,k); };
			// g3d.drawField3D( ctx, rotAngle, fieldFunc, colorMode )
			g3d.drawField3D( ctx, 0.0, densFunc, 1 );
			g3d.drawField3D.threshold = 1;
		} else if (dispMode==5) { // 3D - orbit(x,y,z)
			const energy = theModule.getStEnergy(dispOrbit)
			const occ = theModule.getOccupation(dispOrbit);
			dispText( `orbit(x,y,z):  |${dispOrbit}>,  E=${energy.toFixed(6)},  occ=${occ.toFixed(6)}` );
			const orbitFunc = function(i,j,k) { return 50.0*theModule.getState(dispOrbit, i,j,k); };
			const colorMode = ( occ>0.1 ) ? 1 : 2;
			g3d.drawField3D( ctx, 0.0, orbitFunc, colorMode );
		} else if (dispMode==6) { // 3D - density(x,y,z)
			dispText( "cloud <-- |orbit(x,y,z)|^2" );
			if ( !pauseFlag || inStepFlag ) theModule.setCloud(dispOrbit);
			inStepFlag = false;
			const cloudFunc = function(i,j,k) { return 1.0*theModule.getCloud(i,j,k); };
			g3d.drawField3D( ctx, 0.0, cloudFunc, 1 );

		} else if (dispMode==7) { // 2D - density(x,y,0)
			dispText( "grid2d density(x,y,0)" );
			const densXYFunc = function(i,j) { return rmag*theModule.getDensity(i,j,kc); };
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, densXYFunc, 0.5, 1 );
		} else if (dispMode==8) { // 2D - orbit(x,y,0)
			dispText( "orbit(x,y,z):  |"+dispOrbit+">,  E="+theModule.getStEnergy(dispOrbit).toFixed(6)
						+ ",  occ="+theModule.getOccupation(dispOrbit).toFixed(6) );
			const orbitXYFunc = function(i,j) { return pmag*theModule.getState(dispOrbit,i,j,kc); };
			g3d.drawGrid2D( ctx, 0.0, orbitXYFunc, 0.5, 1  );
		} else if (dispMode==9) { // 2D (density+Vext)(x,y,0)
			dispText( "grid2d (density+Vext)(x,y,0)" );
			const zFunc = function(i,j) { return rmag*theModule.getDensity(i,j,kc) + emag*theModule.getVext(i,j,kc) };
			const colorFunc = function(i,j) {
				const z = rmag*theModule.getDensity(i,j,kc);
				const th = (18120 - Math.floor(180.0*z/g3d.cz0))%360;
				return ( Math.abs(z/g3d.cz0)>0.005 ) ? `hsl(${th},100%,50%)` : `hsl(${th},30%,20%)`;
			}
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 1 );
		} else if (dispMode==10) { // 3D view - Vext(x,y)
			dispText( "grid2d external potential: Vext(x,y,0)" );
			const zFunc = function(i,j) { return emag*theModule.getVext(i,j,kc); };
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, gColor.Vext, 1 );
		} else if (dispMode==11) { // 3D view - Veff(x,y)
			dispText( "grid2d effective potential: Veff(x,y,0)" );
			const zFunc = function(i,j) { return emag*theModule.getVeff(i,j,kc); };
			g3d.drawGrid2D( ctx, 0.0, zFunc, gColor.Veff, 1 );
		} else if (dispMode==12) { // 3D view - Vh(x,y)
			dispText( "grid2d Hartree potential: Vh(x,y,0)" );
			const zFunc = function(i,j) { return emag*theModule.getVh(i,j,kc); };
			g3d.drawGrid2D( ctx, 0.0, zFunc, gColor.Vh, 1 );
		} else if (dispMode==13) { // 3D view - Vxc(x,y)
			dispText( "grid2d exchange and correlation potential: Vxc(x,y,0)" );
			const zFunc = function(i,j) { return emag*(theModule.getVx(i,j,kc)+theModule.getVc(i,j,kc)); };
			g3d.drawGrid2D( ctx, 0.0, zFunc, gColor.Vxc, 1 );
		}

		if ( dispMode<=3 ) { // orbit energy and occupation table
			ctx.fillStyle = "#888888";
			ctx.fillText("orbit   energy(au)", xtabp, 40);
			for (let ist=0; ist<v_stateMax; ist++) {
				const col = `hsl(${ist*30},100%,50%)`;
				ctx.fillStyle = col;
				ctx.fillText(`|${ist}>   ${(theModule.getStEnergy(ist)).toFixed(6)}`, xtabp, 40+(6-ist)*20);
			}
			ctx.fillStyle = "#888888";
			ctx.fillText("orbit  occupation", xtabp, 200);
			for (let ist=0; ist<6; ist++) {
				const occ = theModule.getOccupation(ist);
				ctx.fillStyle = `hsl(${240-occ*120},70%,70%)`;
				ctx.fillText(`|${ist}>   ${(theModule.getOccupation(ist)).toFixed(6)}`, xtabp, 200+(6-ist)*20);
			}

			ctx.fillStyle = "#888888";
			ctx.fillText(`box size : ${(NNx*dx)} (au)`, xtabp, yCanvasSize-50);
			ctx.fillText(`iteration = ${iterCount}`, xtabp, yCanvasSize-30);
			ctx.fillText(`Energy = ${totalEnergy.toFixed(6)}`, xtabp, yCanvasSize-10);
		} else {
			ctx.fillStyle = "#888888";
			ctx.fillText(`box = ${NNx*dx}x${NNy*dx}x${NNz*dx} (au)`, 20, yCanvasSize-30);
			ctx.fillText(`Z = ${p_nucCharge},  ne = ${p_nucCharge+v_dne}`, 260, yCanvasSize-30);
			ctx.fillText(`iteration = ${iterCount}`, 20, yCanvasSize-10);
			ctx.fillText(`Energy = ${totalEnergy.toFixed(6)}`, 260, yCanvasSize-10);
		}

		function dispText( str ) {
			ctx.fillStyle = "#888888";
			ctx.fillText( str, 20, yCanvasSize-50 );
		}
	}

	function draw2D( ctx, ist, xp, yp ) {
		const nnx=p_NNx, nny=p_NNy, nnz=p_NNz, sc=7, kc = p_NNz/2, pmag = 2000.0, rmag = 24000.0;
		ctx.strokeStyle = "#888800";
		ctx.strokeRect( xp, yp, nnx*sc, 256 );

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const g = Math.floor(theModule.getVext(i,j,kc)*10);
				let r,b;
				if (ist>=0) { // orbit
					const p = theModule.getState(ist,i,j,kc)*pmag;
					if (p>=0) {
						r = Math.floor(p); if (r>255) r = 255;
						b = 0;
					} else {
						r=0;
						b = Math.floor(-p); if (b>255) b = 255;
					}
				} else { // density
					const p = theModule.getDensity(i,j,kc)*rmag;
					r = Math.min(Math.floor(p),255);
					b = r;
				}
				ctx.fillStyle = "rgb("+r+","+g+","+b+")";
				ctx.fillRect(i*sc+xp,(nny-j-1)*sc+yp+20,sc,sc);
			}
		}
		ctx.fillStyle = "#888888";
		if (ist>=0) ctx.fillText("orbit - red:ph(x,y)>0 blue:ph(x,y)<0", xp,300);
		if (ist<0) ctx.fillText("density(x,y) - magenta", xp,300);
		ctx.fillText("potential green:Vext(x,y)", xp, 320);
	}

	function draw1D( ctx, mode, xp, yp ) {
		const nnx=p_NNx, nny=p_NNy, nnz=p_NNz, nst=v_stateMax, sc =7;
		const y0 = 220, pmag = 60.0, rmag = 45.0, emag = 15.0, jc = nny/2, kc = nnz/2, tb = 140;

		ctx.strokeStyle = "#888800";
		ctx.strokeRect( xp, yp, nnx*sc, 360 );

		g3d.drawLine( ctx, xp-10, y0, xp+nnx*sc+10, y0, "#444444" ); // base line
		drawFunc( ctx, function(i) { return theModule.getVext(i,jc,kc)*emag; }, xp, y0, gColor.Vext ); // Vext(x,0)
		if ( mode==0 ) drawFunc( ctx, function(i) { return theModule.getDensity(i,jc,kc)*rmag; }, xp, y0, gColor.dens );
		if ( mode==1 ) {
			for (let ist=nst-1; ist>=0; ist--) { // state(x,0)
				const func = function(i) { return theModule.getState(ist,i,jc,kc)*pmag+theModule.getStEnergy(ist)*emag; };
				drawFunc( ctx, func, xp, y0, `hsl(${ist*30},100%,50%)` );
			}
		} else if ( mode==0 ) {
			drawFunc( ctx, function(i) { return theModule.getVeff(i,jc,kc)*emag; }, xp, y0, gColor.Veff );
			drawFunc( ctx, function(i) { return theModule.getVh(i,jc,kc)*emag; }, xp, y0, gColor.Vh );
			drawFunc( ctx, function(i) {
				return (theModule.getVx(i,jc,kc)+theModule.getVc(i,jc,kc))*emag; }, xp, y0, gColor.Vxc );
		}
		if ( mode==0 ) {
			ctx.font = "14px 'sans-serif'";
			ctx.fillStyle = gColor.dens; ctx.fillText("density(x,0,0)", xp, yCanvasSize-70);
			ctx.fillStyle = gColor.Vext; ctx.fillText("Vext(x,0,0)", xp, yCanvasSize-50);
			ctx.fillStyle = gColor.Veff; ctx.fillText("Veff(x,0,0)", xp+tb, yCanvasSize-50);
			ctx.fillStyle = gColor.Vh; ctx.fillText("Vh(x,0,0)", xp, yCanvasSize-30);
			ctx.fillStyle = gColor.Vxc; ctx.fillText("Vxc(x,0,0)", xp+tb, yCanvasSize-30);
			ctx.font = "16px 'sans-serif'";
		} else if ( mode==1 ) {
			ctx.fillStyle = "#888888"; ctx.fillText("orbit(x,0,0)   |0>, |1>, ..., |9>", xp, yCanvasSize-50);
			ctx.fillStyle = gColor.Vext; ctx.fillText("Vext(x,0,0)", xp, yCanvasSize-30);
		}
		ctx.fillStyle = "#888888"; ctx.fillText("along x-axis", xp, yCanvasSize-10);
	}

	function drawFunc( ctx, func, xp, y0, color ) {
		const nnx=p_NNx, nny=p_NNy, sc=7;

		ctx.strokeStyle = color;
		ctx.beginPath();
		for (let i=0; i<nnx; i++) {
			ctx.lineTo(i*sc+xp,y0-func(i));
		}
		ctx.stroke();
	}


	// --------------------  graphics 3D (field) module  --------------------
	//
	// ver 0.0.1  2018.12.16  last updated on 2023.03.01
	// ver 0.0.2  2023.03.03  last updated on 2023.06.01

	let g_NNx, g_NNy, g_NNz, g_dx, g_dy, g_dz, g_xCanvasSize, g_yCanvasSize, g_xBoxSize, g_yShift;

	const g3d = {};				// namespace of graphic 3D module

	g3d.mouseDownFlag = 0;		// 1:on mouse down, 0:else
	g3d.x_mouse = 0;			// x-position of mouse
	g3d.y_mouse = 0;			// y-position of mouse
	g3d.x0_mouse = 0;			// drag-started x-position of mouse
	g3d.y0_mouse = 0;			// drag-started y-position of mouse
	g3d.zoom = 1.0;

	g3d.xMax = 0.0;				// x-length of box
	g3d.yMax = 0.0;				// y-length of box
	g3d.zMax = 0.0;				// z-length of box
	g3d.cx0 = 0.0;				// x-component of rotate center
	g3d.cy0 = 0.0;				// y-component of rotate center
	g3d.cz0 = 0.0;				// z-component of rotate center
	g3d.Ax = -Math.PI/15.0;		// rotate angle around x-axis
	g3d.Ay = -Math.PI/15.0;		// rotate angle around y-axis
	g3d.ddAy = 0.0;				// Ay change rate for auto-rotate: eg. dday=0.5*Math.PI/180
	g3d.cosAx = 0.0;			// cosAx=Math.cos(Ax)
	g3d.sinAx = 0.0;			// sinAx=Math.sin(Ax)
	g3d.cosAy = 0.0;			// cosAy=Math.cos(Ay)
	g3d.sinAy = 0.0;			// sinAy=Math.sin(Ay)

	g3d.xApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.yApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.zApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pxApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pyApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pzApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.boxApex = [[0,0,0], [1,0,0], [0,1,0], [1,1,0], [0,0,1], [1,0,1], [0,1,1], [1,1,1] ];
	g3d.boxEdge = [[0,1,9], [0,2,9], [0,4,9], [1,3,9], [1,5,9], [2,3,9],
					[2,6,9], [3,7,9], [4,5,9], [4,6,9], [5,7,9], [6,7,9] ];

	//--- set mouse on canvas

	// g3d.setMouseOnCanvas( canvas );
	g3d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g3d.mouse_move);
		canvas.addEventListener('mousedown', g3d.mouse_down);
		canvas.addEventListener('mouseup', g3d.mouse_up);
		//canvas.addEventListener("mousewheel", g3d.mouseWheel);
	};

	g3d.mouse_move = function(e) {
		const pi = Math.PI;

		if (g3d.mouseDownFlag==1) {
			g3d.x_mouse = e.clientX;
			g3d.y_mouse = e.clientY;
			g3d.Ay = g3d.Ay + 0.5*(g3d.x_mouse-g3d.x0_mouse)*pi/180;
			if (g3d.Ay<-pi) g3d.Ay += 2*pi;
			if (g3d.Ay>pi) g3d.Ay += -2*pi;
			//g3d.Ax = g3d.Ax + 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			g3d.Ax = g3d.Ax - 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			if (g3d.Ax<-0.5*pi) g3d.Ax = -0.5*pi;
			if (g3d.Ax>0.5*pi) g3d.Ax = 0.5*pi;
			g3d.x0_mouse = g3d.x_mouse;
			g3d.y0_mouse = g3d.y_mouse;
		}
	};

	g3d.mouse_down = function(e) {
		if (g3d.mouseDownFlag==0) {
			g3d.x0_mouse = e.clientX;
			g3d.y0_mouse = e.clientY;
			g3d.x_mouse = g3d.x0_mouse;
			g3d.y_mouse = g3d.y0_mouse;
			g3d.mouseDownFlag = 1;
		}
	};

	g3d.mouse_up = function(e) {
		if (g3d.mouseDownFlag==1) {
			g3d.mouseDownFlag = 0;
		}
	};

	g3d.mouseWheel = function(e) {
		g3d.deltaY = e.deltaY;
		if ( g3d.deltaY > 0 ) g3d.zoom *= 0.95;
		else if ( g3d.deltaY < 0 ) g3d.zoom *= 1.05;
		if ( g3d.zoom<0.5 ) g3d.zoom = 0.5;
		if ( g3d.zoom>2.0 ) g3d.zoom = 2.0;
	};

	//--- 3D graphics aid

	// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
	g3d.init = function( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift ) {
		g_NNx = NNx; g_NNy = NNy; g_NNz = NNz;
		g_dx = dx; g_dy = dx; g_dz = dx;
		g_xCanvasSize = xCanvasSize; g_yCanvasSize = yCanvasSize;
		g_xBoxSize = (xBoxSize==undefined) ? 300 : xBoxSize;
		g_yShift = (yShift==undefined) ? 20 : yShift;
		g3d.setSize();
	}

	g3d.setSize = function() {
		g3d.xMax = g_NNx*g_dx;		// x-length of box
		g3d.yMax = g_NNy*g_dy;		// y-length of box
		g3d.zMax = g_NNz*g_dz;		// z-length of box
		g3d.cx0 = 0.5*g3d.xMax;		// x-component of rotate center
		g3d.cy0 = 0.5*g3d.yMax;		// y-component of rotate center
		g3d.cz0 = 0.5*g3d.zMax;		// z-component of rotate center
	};

	// g3d.setRotateAngle( AxInDegree, AyInDegree );
	g3d.setRotateAngle = function( AxInDegree, AyInDegree ) {
		g3d.Ax = AxInDegree*Math.PI/180.0;
		g3d.Ay = AyInDegree*Math.PI/180.0;
	};

	// g3d.scxpypFunc();
	g3d.scxpypFunc = function() {
		const xBoxSize = g_xBoxSize;
		const xCenter = g_xCanvasSize/2, yCenter = g_yCanvasSize/2-g_yShift, yBoxSize = (xBoxSize/g_NNx)*g_NNy;
		const xp = xCenter - (xBoxSize/2)*g3d.zoom, yp = yCenter - (yBoxSize/2)*g3d.zoom; // g3d param
		const sc = xBoxSize/(g_NNx*g_dx)*g3d.zoom;
		return [ sc, xp, yp ];
	}

	g3d.set3DRotateXY = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate param
		g3d.rotateApexXY();     // box Apex rotate--> pxApex[i],pyApex[i],pzApex[i]
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.setBox = function() {
		for (let i=0; i<8; i++) {
			g3d.xApex[i] = g3d.boxApex[i][0]*g3d.xMax;
			g3d.yApex[i] = g3d.boxApex[i][1]*g3d.yMax;
			g3d.zApex[i] = g3d.boxApex[i][2]*g3d.zMax;
		}
	};

	g3d.setRotateXY = function(angleX,angleY) {
		g3d.cosAx = Math.cos(angleX);
		g3d.sinAx = Math.sin(angleX);
		g3d.cosAy = Math.cos(angleY);
		g3d.sinAy = Math.sin(angleY);
		g3d.cx0 = 0.5*g3d.xMax;
		g3d.cy0 = 0.5*g3d.yMax;
		g3d.cz0 = 0.5*g3d.zMax;
	};

	g3d.rotateApexXY = function() { // rotate box apex
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};

	g3d.markFarEdge = function() {
		//seek far apex --> iMin
		let zMin = g3d.pzApex[0];
		let iMin = 0;
		for (let i=1; i<8; i++) {
			if (zMin>g3d.pzApex[i]) {
				zMin = g3d.pzApex[i];
				iMin = i;
			}
		}
		//mark far edge
		for (let iEdge=0; iEdge<12; iEdge++) {
			g3d.boxEdge[iEdge][2] = 0;
			if (g3d.boxEdge[iEdge][0]==iMin || g3d.boxEdge[iEdge][1]==iMin) g3d.boxEdge[iEdge][2] = 1;
		}
	};

	g3d.drawRotatedDisc = function(ctx, x,y,z,r,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1 = cosAy*(x-cx0)+sinAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cx0
		const y1 = cosAx*(y-cy0)-sinAx*(z-cz0) + cy0
		//z1 =-sinAy*(x-cx0)+cosAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cz0
		g3d.drawDisc(ctx, x1*sc+xp,y1*sc+yp,r,color);
	};

	g3d.drawRotatedLine = function(ctx, x1,y1,z1,x2,y2,z2,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1p = cosAy*(x1-cx0)+sinAy*(sinAx*(y1-cy0)+cosAx*(z1-cz0)) + cx0
		const y1p = cosAx*(y1-cy0)-sinAx*(z1-cz0) + cy0
		const x2p = cosAy*(x2-cx0)+sinAy*(sinAx*(y2-cy0)+cosAx*(z2-cz0)) + cx0
		const y2p = cosAx*(y2-cy0)-sinAx*(z2-cz0) + cy0
		g3d.drawLine(ctx, x1p*sc+xp,y1p*sc+yp,x2p*sc+xp,y2p*sc+yp,color);
	};

	g3d.plotNearEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==0) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotFarEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==1) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotEdge = function(ctx, iEdge,sc,xp,yp,color) {
		let iApex = g3d.boxEdge[iEdge][0];
		const x1=g3d.pxApex[iApex]*sc+xp, y1=g3d.pyApex[iApex]*sc+yp;
		iApex = g3d.boxEdge[iEdge][1];
		const x2=g3d.pxApex[iApex]*sc+xp, y2=g3d.pyApex[iApex]*sc+yp;
		g3d.drawLine(ctx, x1, y1, x2, y2, color);
	};

	g3d.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	g3d.drawDisc = function(ctx, x, y, r, color) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	};

	// --------------------  end of graphics 3D (field) module  --------------------

	// g3d_extension field3D  created 2023.06.04, last updated 2023.06.04
	// g3d.drawField3D( ctx, rotAngle, fieldFunc, colorMode )
	g3d.drawField3D = function(ctx, rotAngle, fieldFunc, colorMode ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dx=g_dx, hh=g_dx*sc;
		const threshold = g3d.drawField3D.threshold;

		g3d.set3DRotateXY(rotAngle);
		g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let ii=0; ii<nnx; ii++) {
			let i=ii; if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj++) {
				let j=jj; if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
				for (let kk=0; kk<nnz; kk++) {
					let k=kk; if (g3d.pzApex[4]-g3d.pzApex[0]<0) k=nnz-kk-1;

					const f = fieldFunc(i,nny-j-1,k);
					const r = Math.min(Math.pow(Math.abs(f),0.333),0.6*hh);
					if ( r>threshold ) {
						let th, colr;
						if ( colorMode==0 ) { // 0:red(f>0) or blue(f<0)
							colr = (f>0) ? "#ff0000" : "#0000ff";
						} else if ( colorMode==1 ) { //
							th = 120.0-30.0*r*Math.sign(f);
							if (th>270.0) th = 270.0;
							if (th<-30.0) th = -30.0;
							colr = `hsl(${th},100%,50%)`;
						} else if ( colorMode==2 ) { // small hsl3-arg.
							th = 120.0-30.0*r*Math.sign(f);
							if (th>270.0) th = 270.0;
							if (th<-30.0) th = -30.0;
							colr = `hsl(${th},60%,30%)`;
						}
						g3d.drawRotatedDisc( ctx, i*dx, j*dx, k*dx, r*g3d.zoom, colr, sc, xp, yp );
					}
				}
			}
		}
		g3d.plotNearEdge( ctx, sc, xp, yp, "#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	g3d.drawField3D.threshold = 0.7;

	// g3d_extension grid2d  created 2023.06.01, last updated 2023.06.04
	// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
	g3d.drawGrid2D = function ( ctx, rotAngle, zFunc, colorFactor, inc, showBox ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx = g_NNx, nny = g_NNy, threshold = g3d.drawGrid2D.threshold;

		g3d.set3DRotateXY(rotAngle);
		if ( (showBox & 1)>0 || showBox==undefined ) g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let jj=0; jj<nny; jj+=inc) {
			let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
			for (let ii=0; ii<nnx; ii++) {
				let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
				if (i<0 || i+1>nnx-1) continue;
				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i+1,nny-j-1);
				const x1 = (i+1)*g_dx, y1 = j*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		for (let ii=0; ii<nnx; ii+=inc) {
			let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj++) {
				let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
				if (j<0 || j+1>nny-1) continue;
				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i,nny-j-2);
				const x1 = i*g_dx, y1 = (j+1)*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		if ( (showBox & 2)>0 || showBox==undefined ) g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	g3d.drawGrid2D.threshold = 0.005;


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
		document.getElementById("slct_orbit").disabled =
			(dispMode==3 || dispMode==5 || dispMode==6 || dispMode==8 ) ? false : true;
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {  // select dNuc
		const dn = 1 + document.getElementById("slct_theme").selectedIndex;
		v_dNuc = 0.5*dn;
		potentialChanged = true;
	}

	function setDzNuc() {  // select v_dzNu
		var sne = 0 + document.getElementById("slct_dne").selectedIndex;
		var dne = 0.0;

		if (sne==0) { dne = -1.0; }      // +ion
		else if (sne==1) { dne = -0.5; } // ionization potential
		else if (sne==2) { dne = 0.0; }  // atom
		else if (sne==3) { dne = 1.0; }  // -ion
		v_dne = dne;
		potentialChanged = true;
	}

	function setChargeMixing() {  // range mixing
		v_mix = Number(document.getElementById("range_mixing").value);
		document.getElementById("text_mixing").innerHTML = " " + v_mix.toFixed(3);
	}

	function setLevelBroadening() {  // range broadening
		v_broad = Number(document.getElementById("range_broadening").value);
		document.getElementById("text_broadening").innerHTML = " " + v_broad.toFixed(3);
	}

	function setDispMode() {  // select dispMode
		const dom = document.getElementById("slct_orbit");
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
		dom.disabled = (dispMode==3 || dispMode==5 || dispMode==6 || dispMode==8 ) ? false : true;
	}

	function setOrbit() {  // select dispOrbit
		dispOrbit = 0 + document.getElementById("slct_orbit").selectedIndex;
	}

	function viewHome() {
		g3d.setRotateAngle(65,-15);
		g3d.zoom = 1.0;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( dn ) { // dn = 1,2,3,4,5,6
		v_dNuc = 0.5*dn;
		potentialChanged = true;
    document.getElementById("slct_theme").selectedIndex = (dn-1);
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pysetDispState( ist ) {
    dispState = ist;
    document.getElementById("slct_dispState").selectedIndex = ist;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ iterCount, totalEnergy, occList, energyList ];
  }

  function pygetFieldData() {
		fieldKind = 0;
    return [ nowData, vextArray, veffArray, densityArray, orb0Array, orb1Array, orb2Array, orb3Array ];
  }


	// --------------------  public  --------------------

	return {
		main:				main,				// main()

		reset:				reset,				// reset()
		pause:				pause,				// pause()
		step:				step,				// step()

		setTheme:			setTheme,			// setTheme()
		setDzNuc:			setDzNuc,			// setDzNuc()
		setChargeMixing:	setChargeMixing,	// setChargeMixing()
		setLevelBroadening:	setLevelBroadening,	// setLevelBroadening()
		setDispMode:		setDispMode,		// setDispMode()
		setOrbit:			setOrbit,			// setOrbit()
		viewHome:			viewHome,			// viewHome()

    breakLoop: breakLoop, // breakLoop();
		pysetTheme: pysetTheme, // pysetTheme( dn ) // dn = 1,2,..,6
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
		pysetDispState: pysetDispState, // pysetDispState( ist )
		pygetData: pygetData, // pygetData( pyMsg ) : return [ iterCount, totalEnergy, occList, energyList ]
	  pygetFieldData: pygetFieldData, // pygetFieldData() : return [ nowData, vextArray, veffArray, densityArray, orb0Array, orb1Array, orb2Array, orb3Array ]
	};

})(); // ====================  js043 end  ====================

const js = js043;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js043] H2 molecule - Real Space Density Functional Theory LDA3D</p>
<canvas ID="canvas_box" style="background-color: #000000;" WIDTH="480" HEIGHT="480"></canvas>
<br>

<label>H-H distance =</label>
<select id="slct_theme" onChange="js.setTheme()">
<option>0.5</option><option>1.0</option><option selected>1.5</option>
<option>2.0</option><option>2.5</option><option>3.0</option>
</select>
(au)
    <span style="margin-right: 10px;"></span>
<label>status:</label>
<select id="slct_dne" onChange="js.setDzNuc()">
<option>+ion</option><option>ionization</option><option selected>molecule</option>
<option>-ion</option></select>
    <span style="margin-right: 20px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>mixing:</label><label id="text_mixing"> 0.500</label>
<input type="range" id="range_mixing" min="0.00" max="0.9" value="0.50" step="0.01"
style="width:360px" list="tickmarks_mx" oninput="js.setChargeMixing()">
<datalist id="tickmarks_mx"><option value="0.2"><option value="0.4"><option value="0.6">
<option value="0.8"></datalist>
<br>

<label>broad :</label><label id="text_broadening"> 0.010</label>
<input type="range" id="range_broadening" min="0.001" max="0.1" value="0.01" step="0.001"
style="width:360px" list="tickmarks_br" oninput="js.setLevelBroadening()">
<datalist id="tickmarks_br"><option value="0.02"><option value="0.04"><option value="0.06">
<option value="0.08"><option value="0.1"></datalist>
<br>

<label>disp mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option>1d: density(x,0,0) + Vext,Veff,Vh,Vxc(x,0,0)</option><option>1d: orbit(x,0,0) + Vext(x,0,0)</option>
<option>2d: density(x,y,0)</option><option>2d: orbit(x,y,0)</option>
<option selected>3d: density(x,y,z)</option><option>3d: orbit(x,y,z)</option><option>3d: cloud(|orbit|^2)</option>
<option>grid2d: density(x,y,0)</option><option>grid2d: orbit(x,y,0)</option>
<option>grid2d: (density+Vext)(x,y,0)</option>
<option>grid2d: Vext(x,y,0)</option><option>grid2d: Veff(x,y,0)</option>
<option>grid2d: Vh(x,y,0)</option><option>grid2d: Vxc(x,y,0)</option>
</select>
    <span style="margin-right: 20px;"></span>
<label>orbit:</label>
<select id="slct_orbit" onChange="js.setOrbit()">
<option selected>0</option><option>1</option><option>2</option><option>3</option><option>4</option>
<option>5</option>
</select>
<br>

<button onClick="js.viewHome()">return to initial view</button>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# gat data and print

import time

# simulator run
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  [ iterCount, totalEnergy, occList, energyList  ] = eval_js( 'js.pygetData({})'.format(i) )
  print(f'i = {i:>2},  iter count = {iterCount:>4},  total energy = {totalEnergy:>10.6f} (au)')
  time.sleep(1)

stateList = range(3)
stateOccEnergyList = [f'|{st:>2d} > occ={occ:>5.3f}, energy={energy:>10.6f}' for st, occ, energy in zip(stateList, occList, energyList) ]
print( "state occupation energy :", stateOccEnergyList )

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- stop --")

In [ ]:
# change dispMode

import time

dispModeList = [
    '0: 1d-density(x,0,0) + Vext(x,0,0)', '1:1d- orbit(x,0,0) + Vext(x,0,0)', '2: 2d-density(x,y,0)', '3: 2d-orbit(x,y,0)',
    '4: 3d-density(x,y,z)', '5: 3d-orbit(x,y,z)', '6: 3d-cloud', '7: grid2d-density(x,y,0)', '8: grid2d-orbit(x,y,0)', '9: grid2d-density+Vext',
    '10: grid2d-Vext(x,y,0)', '11: grid2d-Veff(x,y,0)', '12: grid2d-Vh(x,y,0)',  '13: grid2d-Vxc(x,y,0)' ]

# simulator run
exec_html_js()
print("-- start --")

# get data and print
for dispMode in range(len(dispModeList)):
  eval_js( 'js.pysetDispMode({})'.format(dispMode) )
  print("-- dispMode:",dispModeList[dispMode], " --")
  [ iterCount, totalEnergy, occList, energyList ] = eval_js( 'js.pygetData({})'.format(i) )
  print(f"\t iteration count = {iterCount:>4d},  total energy = {totalEnergy:>10.6f} ")
  time.sleep(3)

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- stop --")

In [ ]:
# total energy vs H-H distance

import time
import matplotlib.pyplot as plt

dHH_list = []
energy_list = []

# simulator run
exec_html_js()
print("-- start --")

# python control
for i in [ 1, 2, 3, 4, 5, 6 ]:
  eval_js( 'js.pysetTheme({})'.format(i) )
  dHH = i*0.5
  print("-- H-H distance =",dHH," (au) --" )
  [ iterCount, totalEnergy, occList, energyList ] = eval_js( 'js.pygetData({})'.format(i) )
  print(f"\t iteration count = {iterCount:>4d},  total energy = {totalEnergy:>10.6f} ")
  time.sleep(5)
  [ iterCount, totalEnergy, occList, energyList ] = eval_js( 'js.pygetData({})'.format(i) )
  print(f"\t iteration count = {iterCount:>4d},  total energy = {totalEnergy:>10.6f} ")

  dHH_list.append(dHH)
  energy_list.append(totalEnergy)

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- simulator stop --")

# plot dHH - total energy
plt.plot( dHH_list, energy_list, 'or-' )
plt.xlabel("d(H-H) in (au)")
plt.ylabel("total energy (au)")
plt.show()

In [ ]:
# get field data

import time
import numpy as np

# simulator run
exec_html_js()
print("-- start --")

# get data and print
for i in range(6):
  [ iterCount, totalEnergy, occList, energyList ] = eval_js( 'js.pygetData({})'.format(i) )
  print(f'i = {i:>2},  iter count = {iterCount:>4},  total energy = {totalEnergy:>10.6f} (au)')
  time.sleep(3)


# get field data
print("-- get field data --\n wait a minute ..... ")
[ nowData, vextArray, veffArray, densityArray, orb0Array, orb1Array, orb2Array, orb3Array ] = eval_js( 'js.pygetFieldData()' )
[ gotCount, totalEnergy, occList, energyList ] = nowData
print(f"\t got count = {gotCount:>4d},  total energy = {totalEnergy:>10.6f} ")
stateList = range(3)
stateOccEnergyList = [f'|{st:>2d} > occ={occ:>5.3f}, energy={energy:>10.6f}' for st, occ, energy in zip(stateList, occList, energyList) ]
print( "state occupation energy :", stateOccEnergyList )

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- simulator stop --")

# pack data
occ_energy_data = np.array([ occList, energyList ])
field_data = np.array([ vextArray, veffArray, densityArray, orb0Array, orb1Array, orb2Array, orb3Array ])
print(f'occ_energy data shape:{occ_energy_data.shape},  field data shape:{field_data.shape}')

# save data
np.save( 'js043_occ_energy_data.npy', occ_energy_data )
np.save( 'js043_field_data.npy', field_data )
print("-- saved --")

In [ ]:
# load data

import numpy as np

occ_energy_data = np.load( 'js043_occ_energy_data.npy')
field_data = np.load( 'js043_field_data.npy')
print(f'occ_energy data shape:{occ_energy_data.shape},  field data shape:{field_data.shape}')

# set numpy array
Occ = occ_energy_data[0]
OrbEnergy = occ_energy_data[1]
print(f" Occ :{Occ} \n OrbEnergy :{OrbEnergy}")
Vext = field_data[0]
Veff = field_data[1]
Dens = field_data[2]
Orb = field_data[3:]
print(f"Vext shape :{Vext.shape} , Veff shape :{Veff.shape}, Dens shape :{Dens.shape}, Orb shape :{Orb.shape}")

In [ ]:
# electron orbit_0(x,y,0) / image plot

import numpy as np
import matplotlib.pyplot as plt

Orb0 = Orb[0]
nx, ny, nz = Orb0.shape

Z = Orb0[ : , : , int(nz/2)].T
img = plt.imshow(Z, origin='lower', cmap='jet' )
plt.colorbar(img)
plt.title("orbit_0(x,y,0)] map")
plt.show()

In [ ]:
# prompt: plot Orb[0], .., Orb[3] in 2x2 subplot

# electron orbit_0(x,y,0) / image plot


Orb0 = Orb[0]
Orb1 = Orb[1]
Orb2 = Orb[2]
Orb3 = Orb[3]
nx, ny, nz = Orb0.shape

Z0 = Orb0[ : , : , int(nz/2)].T
Z1 = Orb1[ : , : , int(nz/2)].T
Z2 = Orb2[ : , : , int(nz/2)].T
Z3 = Orb3[ : , : , int(nz/2)].T

fig, axes = plt.subplots(2, 2, figsize=(10, 10))
ax = axes.ravel()

ax[0].imshow(Z0, origin='lower', cmap='jet' )
ax[0].set_title("orbit_0(x,y,0)")
ax[1].imshow(Z1, origin='lower', cmap='jet' )
ax[1].set_title("orbit_1(x,y,0)")
ax[2].imshow(Z2, origin='lower', cmap='jet' )
ax[2].set_title("orbit_2(x,y,0)")
ax[3].imshow(Z3, origin='lower', cmap='jet' )
ax[3].set_title("orbit_3(x,y,0)")

plt.tight_layout()
plt.show()


In [ ]:
# plot external potential Vext(x,y,0) - surface plot

import numpy as np
import matplotlib.pyplot as plt

def surface_plot( FieldArray3d, titleString ):
  nx, ny, nz = FieldArray3d.shape
  Field2d = FieldArray3d[ : , : , int(nz/2) ]

  # surface plot with adjusted aspect ratio
  fig = plt.figure(figsize=(8,8))
  ax = fig.add_subplot(projection='3d')

  # Plot the surface
  x = np.arange(-nx/2,nx/2)
  y = np.arange(-ny/2,ny/2)
  X, Y = np.meshgrid(x, y)
  Z = Field2d.T
  ax.plot_surface(X,Y,Z, cmap='jet')

  # Adjust aspect ratio
  ax.set_box_aspect([1, ny/nx, 0.7])  # [x, y, z] aspect ratios
  plt.xlabel("x")
  plt.ylabel("y")
  plt.title( titleString )
  plt.show()

surface_plot( Vext, 'external potential Vext(x,y,0)' )

In [ ]:
# plot electron density dens(x,y,0) - surface plot
# loaded_data = np.array([ vextArray, veffArray, densityArray, orb0Array, orb1Array, orb2Array ])

surface_plot( Dens, 'electron density dens(x,y,0)' )

In [ ]:
# plot orbit_1(x,y,0) - surface plot
# loaded_data = np.array([ vextArray, veffArray, densityArray, orb0Array, orb1Array, orb2Array ])

surface_plot( Orb[1], 'Orbit_1(x,y,0) : occ=0' )

In [ ]:
# colab AI wrote:
# prompt: calc. inner product matrix of Orb[0], .. , Orb[3] , dx=dy=dz= 1/2

import numpy as np

dx = 0.5
dy = 0.5
dz = 0.5

# initialize the inner product matrix
inner_product_matrix = np.zeros((4, 4))

# calculate the inner product of each pair of orbitals
for i in range(4):
  for j in range(4):
    inner_product = np.sum(Orb[i] * Orb[j] * dx * dy * dz)
    inner_product_matrix[i, j] = inner_product

# print the inner product matrix
print("Inner product matrix:")
print(inner_product_matrix)


- TODO<br>
3d plot of Dens
